# Evolver Loop 3 Analysis

## Key Questions:
1. Can per-target heterogeneous models improve performance?
2. What's the impact of combining multiple feature sets (acs_pca + spange)?
3. What's the gap between our best CV (0.0805) and target (0.0173)?
4. What approaches from public kernels haven't been tried?

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, HistGradientBoostingRegressor
from sklearn.model_selection import LeaveOneGroupOut
import warnings
warnings.filterwarnings('ignore')

# Load data
DATA_PATH = '/home/data'
df_single = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
df_full = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
spange = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
acs_pca = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)

TARGET_LABELS = ['Product 2', 'Product 3', 'SM']
print(f'Single solvent: {df_single.shape}')
print(f'Full data: {df_full.shape}')
print(f'Spange: {spange.shape}')
print(f'ACS PCA: {acs_pca.shape}')

Single solvent: (656, 13)
Full data: (1227, 19)
Spange: (26, 13)
ACS PCA: (24, 5)


In [2]:
# Test the dabansherwani approach: per-target heterogeneous models
# SM -> HistGradientBoosting
# Products -> ExtraTrees

print('=== Testing Per-Target Heterogeneous Models (dabansherwani approach) ===')

# Prepare features - Spange only first
df_test = df_single.copy()
for col in spange.columns:
    df_test[f'spange_{col}'] = df_test['SOLVENT NAME'].map(spange[col])

# Add process features
df_test['inv_temp'] = 1000 / (df_test['Temperature'] + 273.15)
df_test['log_time'] = np.log(df_test['Residence Time'] + 1e-6)
df_test['interaction'] = df_test['inv_temp'] * df_test['log_time']

process_cols = ['Residence Time', 'Temperature', 'inv_temp', 'log_time', 'interaction']
spange_cols = [f'spange_{col}' for col in spange.columns]
feature_cols = process_cols + spange_cols

X = df_test[feature_cols].values
y = df_test[TARGET_LABELS].values
groups = df_test['SOLVENT NAME'].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

logo = LeaveOneGroupOut()

# Per-target heterogeneous model
per_target_errors = []
for train_idx, test_idx in logo.split(X_scaled, y, groups):
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    preds = np.zeros_like(y_test)
    
    # SM -> HistGradientBoosting
    hgb = HistGradientBoostingRegressor(max_depth=7, max_iter=700, learning_rate=0.04, random_state=42)
    hgb.fit(X_train, y_train[:, 2])  # SM is index 2
    preds[:, 2] = hgb.predict(X_test)
    
    # Products -> ExtraTrees
    for t in [0, 1]:  # Product 2, Product 3
        etr = ExtraTreesRegressor(n_estimators=900, min_samples_leaf=2, random_state=42, n_jobs=-1)
        etr.fit(X_train, y_train[:, t])
        preds[:, t] = etr.predict(X_test)
    
    preds = np.clip(preds, 0, 1)
    mae = np.mean(np.abs(preds - y_test))
    per_target_errors.append(mae)

print(f'Per-Target Heterogeneous (Spange only) MAE: {np.mean(per_target_errors):.4f} +/- {np.std(per_target_errors):.4f}')

=== Testing Per-Target Heterogeneous Models (dabansherwani approach) ===


Per-Target Heterogeneous (Spange only) MAE: 0.0755 +/- 0.0350


In [3]:
# Now test with combined features (acs_pca + spange) with weighted averaging
print('\n=== Testing Combined Features (acs_pca + spange) with Weighted Averaging ===')

# Prepare acs_pca features
df_test_acs = df_single.copy()
for col in acs_pca.columns:
    df_test_acs[f'acs_{col}'] = df_test_acs['SOLVENT NAME'].map(acs_pca[col])

# Add process features
df_test_acs['inv_temp'] = 1000 / (df_test_acs['Temperature'] + 273.15)
df_test_acs['log_time'] = np.log(df_test_acs['Residence Time'] + 1e-6)
df_test_acs['interaction'] = df_test_acs['inv_temp'] * df_test_acs['log_time']

acs_cols = [f'acs_{col}' for col in acs_pca.columns]
feature_cols_acs = process_cols + acs_cols

X_acs = df_test_acs[feature_cols_acs].values
scaler_acs = StandardScaler()
X_acs_scaled = scaler_acs.fit_transform(X_acs)

# Per-target with weighted averaging of two feature sets
combined_errors = []
for train_idx, test_idx in logo.split(X_scaled, y, groups):
    X_train_sp, X_test_sp = X_scaled[train_idx], X_scaled[test_idx]
    X_train_acs, X_test_acs = X_acs_scaled[train_idx], X_acs_scaled[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    preds_sp = np.zeros_like(y_test)
    preds_acs = np.zeros_like(y_test)
    
    # SM -> HistGradientBoosting on both feature sets
    hgb_sp = HistGradientBoostingRegressor(max_depth=7, max_iter=700, learning_rate=0.04, random_state=42)
    hgb_sp.fit(X_train_sp, y_train[:, 2])
    preds_sp[:, 2] = hgb_sp.predict(X_test_sp)
    
    hgb_acs = HistGradientBoostingRegressor(max_depth=7, max_iter=700, learning_rate=0.04, random_state=42)
    hgb_acs.fit(X_train_acs, y_train[:, 2])
    preds_acs[:, 2] = hgb_acs.predict(X_test_acs)
    
    # Products -> ExtraTrees on both feature sets
    for t in [0, 1]:
        etr_sp = ExtraTreesRegressor(n_estimators=900, min_samples_leaf=2, random_state=42, n_jobs=-1)
        etr_sp.fit(X_train_sp, y_train[:, t])
        preds_sp[:, t] = etr_sp.predict(X_test_sp)
        
        etr_acs = ExtraTreesRegressor(n_estimators=900, min_samples_leaf=2, random_state=42, n_jobs=-1)
        etr_acs.fit(X_train_acs, y_train[:, t])
        preds_acs[:, t] = etr_acs.predict(X_test_acs)
    
    # Weighted average (0.65 acs_pca + 0.35 spange as per dabansherwani)
    preds = 0.65 * preds_acs + 0.35 * preds_sp
    preds = np.clip(preds, 0, 1)
    
    mae = np.mean(np.abs(preds - y_test))
    combined_errors.append(mae)

print(f'Combined Features (0.65 acs + 0.35 spange) MAE: {np.mean(combined_errors):.4f} +/- {np.std(combined_errors):.4f}')


=== Testing Combined Features (acs_pca + spange) with Weighted Averaging ===


Combined Features (0.65 acs + 0.35 spange) MAE: 0.0670 +/- 0.0322


In [4]:
# Compare all approaches
print('\n=== Comparison of Approaches ===')
print(f'Our best (exp_003 RF): 0.0748 (single solvent)')
print(f'Per-Target Heterogeneous (Spange): {np.mean(per_target_errors):.4f}')
print(f'Per-Target + Combined Features: {np.mean(combined_errors):.4f}')
print(f'Target: 0.0173')
print(f'\nGap to target: {np.mean(combined_errors) - 0.0173:.4f} ({(np.mean(combined_errors)/0.0173):.1f}x)')


=== Comparison of Approaches ===
Our best (exp_003 RF): 0.0748 (single solvent)
Per-Target Heterogeneous (Spange): 0.0755
Per-Target + Combined Features: 0.0670
Target: 0.0173

Gap to target: 0.0497 (3.9x)


In [5]:
# Test different weight combinations
print('\n=== Testing Different Weight Combinations ===')

best_weight = None
best_mae = float('inf')

for w_acs in [0.3, 0.4, 0.5, 0.6, 0.7, 0.8]:
    w_sp = 1 - w_acs
    
    weight_errors = []
    for train_idx, test_idx in logo.split(X_scaled, y, groups):
        X_train_sp, X_test_sp = X_scaled[train_idx], X_scaled[test_idx]
        X_train_acs, X_test_acs = X_acs_scaled[train_idx], X_acs_scaled[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        preds_sp = np.zeros_like(y_test)
        preds_acs = np.zeros_like(y_test)
        
        # SM -> HistGradientBoosting
        hgb_sp = HistGradientBoostingRegressor(max_depth=7, max_iter=700, learning_rate=0.04, random_state=42)
        hgb_sp.fit(X_train_sp, y_train[:, 2])
        preds_sp[:, 2] = hgb_sp.predict(X_test_sp)
        
        hgb_acs = HistGradientBoostingRegressor(max_depth=7, max_iter=700, learning_rate=0.04, random_state=42)
        hgb_acs.fit(X_train_acs, y_train[:, 2])
        preds_acs[:, 2] = hgb_acs.predict(X_test_acs)
        
        # Products -> ExtraTrees
        for t in [0, 1]:
            etr_sp = ExtraTreesRegressor(n_estimators=500, min_samples_leaf=2, random_state=42, n_jobs=-1)
            etr_sp.fit(X_train_sp, y_train[:, t])
            preds_sp[:, t] = etr_sp.predict(X_test_sp)
            
            etr_acs = ExtraTreesRegressor(n_estimators=500, min_samples_leaf=2, random_state=42, n_jobs=-1)
            etr_acs.fit(X_train_acs, y_train[:, t])
            preds_acs[:, t] = etr_acs.predict(X_test_acs)
        
        preds = w_acs * preds_acs + w_sp * preds_sp
        preds = np.clip(preds, 0, 1)
        
        mae = np.mean(np.abs(preds - y_test))
        weight_errors.append(mae)
    
    mean_mae = np.mean(weight_errors)
    print(f'Weight acs={w_acs:.1f}, spange={w_sp:.1f}: MAE = {mean_mae:.4f}')
    
    if mean_mae < best_mae:
        best_mae = mean_mae
        best_weight = w_acs

print(f'\nBest weight: acs={best_weight:.1f}, spange={1-best_weight:.1f} with MAE={best_mae:.4f}')


=== Testing Different Weight Combinations ===


Weight acs=0.3, spange=0.7: MAE = 0.0709


Weight acs=0.4, spange=0.6: MAE = 0.0697


Weight acs=0.5, spange=0.5: MAE = 0.0685


Weight acs=0.6, spange=0.4: MAE = 0.0676


Weight acs=0.7, spange=0.3: MAE = 0.0668


Weight acs=0.8, spange=0.2: MAE = 0.0662

Best weight: acs=0.8, spange=0.2 with MAE=0.0662


In [6]:
# Test simpler models with combined features
print('\n=== Testing Simpler Models with Combined Features ===')

# Concatenate both feature sets
X_combined = np.hstack([X_scaled, X_acs_scaled[:, len(process_cols):]])  # Avoid duplicating process cols
print(f'Combined feature shape: {X_combined.shape}')

# Test RF with combined features
rf_combined_errors = []
for train_idx, test_idx in logo.split(X_combined, y, groups):
    X_train, X_test = X_combined[train_idx], X_combined[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    rf = RandomForestRegressor(n_estimators=200, max_depth=8, min_samples_leaf=5, random_state=42, n_jobs=-1)
    rf.fit(X_train, y_train)
    preds = rf.predict(X_test)
    preds = np.clip(preds, 0, 1)
    
    mae = np.mean(np.abs(preds - y_test))
    rf_combined_errors.append(mae)

print(f'RF with Combined Features MAE: {np.mean(rf_combined_errors):.4f} +/- {np.std(rf_combined_errors):.4f}')


=== Testing Simpler Models with Combined Features ===
Combined feature shape: (656, 23)


RF with Combined Features MAE: 0.0746 +/- 0.0351


In [7]:
# Test ExtraTrees with different regularization
print('\n=== Testing ExtraTrees with Different Regularization ===')

for max_depth in [5, 8, 10, 15, None]:
    for min_samples_leaf in [2, 5, 10]:
        etr_errors = []
        for train_idx, test_idx in logo.split(X_scaled, y, groups):
            X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            
            etr = ExtraTreesRegressor(n_estimators=200, max_depth=max_depth, min_samples_leaf=min_samples_leaf, random_state=42, n_jobs=-1)
            etr.fit(X_train, y_train)
            preds = etr.predict(X_test)
            preds = np.clip(preds, 0, 1)
            
            mae = np.mean(np.abs(preds - y_test))
            etr_errors.append(mae)
        
        print(f'ETR max_depth={max_depth}, min_samples_leaf={min_samples_leaf}: MAE = {np.mean(etr_errors):.4f}')


=== Testing ExtraTrees with Different Regularization ===


ETR max_depth=5, min_samples_leaf=2: MAE = 0.0722


ETR max_depth=5, min_samples_leaf=5: MAE = 0.0727


ETR max_depth=5, min_samples_leaf=10: MAE = 0.0740


ETR max_depth=8, min_samples_leaf=2: MAE = 0.0693


...

ETR max_depth=15, min_samples_leaf=5: MAE = 0.0712


ETR max_depth=15, min_samples_leaf=10: MAE = 0.0741


ETR max_depth=None, min_samples_leaf=2: MAE = 0.0696


ETR max_depth=None, min_samples_leaf=5: MAE = 0.0711


ETR max_depth=None, min_samples_leaf=10: MAE = 0.0741


In [8]:
# Summary
print('\n=== SUMMARY ===')
print('\nKey findings:')
print('1. Per-target heterogeneous models (HGB for SM, ETR for Products) show promise')
print('2. Combining acs_pca + spange features may help')
print('3. The gap to target (0.0173) is still ~4-5x')
print('\nRecommendations for next experiment:')
print('1. Implement per-target heterogeneous model with combined features')
print('2. Try different weight combinations for feature set averaging')
print('3. Consider more aggressive regularization for tree models')
print('4. The target may require fundamentally different approaches (transformers, GPs)')


=== SUMMARY ===

Key findings:
1. Per-target heterogeneous models (HGB for SM, ETR for Products) show promise
2. Combining acs_pca + spange features may help
3. The gap to target (0.0173) is still ~4-5x

Recommendations for next experiment:
1. Implement per-target heterogeneous model with combined features
2. Try different weight combinations for feature set averaging
3. Consider more aggressive regularization for tree models
4. The target may require fundamentally different approaches (transformers, GPs)


In [9]:
# Test on full data (mixed solvents) as well
print('\n=== Testing on Full Data (Mixed Solvents) ===')

# Prepare features for full data
df_full_test = df_full.copy()

# Add spange features for mixed solvents
for col in spange.columns:
    df_full_test[f'spange_A_{col}'] = df_full_test['SOLVENT A NAME'].map(spange[col])
    df_full_test[f'spange_B_{col}'] = df_full_test['SOLVENT B NAME'].map(spange[col])

# Create mixed features
for col in spange.columns:
    df_full_test[f'spange_mix_{col}'] = (
        df_full_test[f'spange_A_{col}'] * (1 - df_full_test['SolventB%']/100) +
        df_full_test[f'spange_B_{col}'] * (df_full_test['SolventB%']/100)
    )

# Add process features
df_full_test['inv_temp'] = 1000 / (df_full_test['Temperature'] + 273.15)
df_full_test['log_time'] = np.log(df_full_test['Residence Time'] + 1e-6)
df_full_test['interaction'] = df_full_test['inv_temp'] * df_full_test['log_time']

process_cols_full = ['Residence Time', 'Temperature', 'SolventB%', 'inv_temp', 'log_time', 'interaction']
spange_mix_cols = [f'spange_mix_{col}' for col in spange.columns]
feature_cols_full = process_cols_full + spange_mix_cols

X_full = df_full_test[feature_cols_full].values
y_full = df_full_test[TARGET_LABELS].values

# Create groups for leave-one-ramp-out
df_full_test['ramp'] = df_full_test['SOLVENT A NAME'] + '_' + df_full_test['SOLVENT B NAME']
groups_full = df_full_test['ramp'].values

scaler_full = StandardScaler()
X_full_scaled = scaler_full.fit_transform(X_full)

# Test per-target heterogeneous on full data
logo_full = LeaveOneGroupOut()
full_errors = []
for train_idx, test_idx in logo_full.split(X_full_scaled, y_full, groups_full):
    X_train, X_test = X_full_scaled[train_idx], X_full_scaled[test_idx]
    y_train, y_test = y_full[train_idx], y_full[test_idx]
    
    preds = np.zeros_like(y_test)
    
    # SM -> HistGradientBoosting
    hgb = HistGradientBoostingRegressor(max_depth=7, max_iter=700, learning_rate=0.04, random_state=42)
    hgb.fit(X_train, y_train[:, 2])
    preds[:, 2] = hgb.predict(X_test)
    
    # Products -> ExtraTrees
    for t in [0, 1]:
        etr = ExtraTreesRegressor(n_estimators=500, min_samples_leaf=2, max_depth=10, random_state=42, n_jobs=-1)
        etr.fit(X_train, y_train[:, t])
        preds[:, t] = etr.predict(X_test)
    
    preds = np.clip(preds, 0, 1)
    mae = np.mean(np.abs(preds - y_test))
    full_errors.append(mae)

print(f'Per-Target Heterogeneous (Full Data) MAE: {np.mean(full_errors):.4f} +/- {np.std(full_errors):.4f}')


=== Testing on Full Data (Mixed Solvents) ===


Per-Target Heterogeneous (Full Data) MAE: 0.0919 +/- 0.0245


In [10]:
# Test with TTA for mixed solvents (flip A and B)\nprint('\\n=== Testing with TTA for Mixed Solvents ===')\n\n# Test per-target heterogeneous with TTA\nfull_tta_errors = []\nfor train_idx, test_idx in logo_full.split(X_full_scaled, y_full, groups_full):\n    X_train, X_test = X_full_scaled[train_idx], X_full_scaled[test_idx]\n    y_train, y_test = y_full[train_idx], y_full[test_idx]\n    \n    # Also create flipped training data\n    df_train = df_full_test.iloc[train_idx].copy()\n    df_test = df_full_test.iloc[test_idx].copy()\n    \n    # Create flipped features for training\n    df_train_flip = df_train.copy()\n    for col in spange.columns:\n        df_train_flip[f'spange_mix_{col}'] = (\n            df_train[f'spange_B_{col}'] * (1 - df_train['SolventB%']/100) +\n            df_train[f'spange_A_{col}'] * (df_train['SolventB%']/100)\n        )\n    \n    X_train_flip = df_train_flip[feature_cols_full].values\n    X_train_flip_scaled = scaler_full.transform(X_train_flip)\n    \n    # Augment training data\n    X_train_aug = np.vstack([X_train, X_train_flip_scaled])\n    y_train_aug = np.vstack([y_train, y_train])\n    \n    preds = np.zeros_like(y_test)\n    \n    # SM -> HistGradientBoosting\n    hgb = HistGradientBoostingRegressor(max_depth=7, max_iter=700, learning_rate=0.04, random_state=42)\n    hgb.fit(X_train_aug, y_train_aug[:, 2])\n    preds[:, 2] = hgb.predict(X_test)\n    \n    # Products -> ExtraTrees\n    for t in [0, 1]:\n        etr = ExtraTreesRegressor(n_estimators=500, min_samples_leaf=2, max_depth=10, random_state=42, n_jobs=-1)\n        etr.fit(X_train_aug, y_train_aug[:, t])\n        preds[:, t] = etr.predict(X_test)\n    \n    # TTA: also predict with flipped test features\n    df_test_flip = df_test.copy()\n    for col in spange.columns:\n        df_test_flip[f'spange_mix_{col}'] = (\n            df_test[f'spange_B_{col}'] * (1 - df_test['SolventB%']/100) +\n            df_test[f'spange_A_{col}'] * (df_test['SolventB%']/100)\n        )\n    X_test_flip = df_test_flip[feature_cols_full].values\n    X_test_flip_scaled = scaler_full.transform(X_test_flip)\n    \n    preds_flip = np.zeros_like(y_test)\n    preds_flip[:, 2] = hgb.predict(X_test_flip_scaled)\n    for t in [0, 1]:\n        etr = ExtraTreesRegressor(n_estimators=500, min_samples_leaf=2, max_depth=10, random_state=42, n_jobs=-1)\n        etr.fit(X_train_aug, y_train_aug[:, t])\n        preds_flip[:, t] = etr.predict(X_test_flip_scaled)\n    \n    # Average predictions\n    preds = (preds + preds_flip) / 2\n    preds = np.clip(preds, 0, 1)\n    \n    mae = np.mean(np.abs(preds - y_test))\n    full_tta_errors.append(mae)\n\nprint(f'Per-Target Heterogeneous with TTA (Full Data) MAE: {np.mean(full_tta_errors):.4f} +/- {np.std(full_tta_errors):.4f}')

In [11]:
# Final summary with combined score estimate\nprint('\\n=== FINAL SUMMARY ===')\nprint('\\nSingle Solvent Results:')\nprint(f'  Current best (exp_003 RF): 0.0748')\nprint(f'  Per-Target Heterogeneous + Combined Features: 0.0662')\nprint(f'  Improvement: {(0.0748 - 0.0662)/0.0748 * 100:.1f}%')\n\nprint('\\nFull Data Results:')\nprint(f'  Current best (exp_003 RF): 0.0836')\nprint(f'  Per-Target Heterogeneous: 0.0919 (worse without TTA)')\n\nprint('\\nRecommendation:')\nprint('  Implement per-target heterogeneous model with:')\nprint('  - Combined features (0.8 acs_pca + 0.2 spange)')\nprint('  - HGB for SM, ETR for Products')\nprint('  - TTA for mixed solvents')\nprint('  - Expected combined score: ~0.075 (vs current 0.0805)')"